In [3]:
import h5py
import pandas as pd
import os

from tqdm import tqdm

In [4]:
def convert_to_hdf5(input_dir, output_file):
    file_names = [f for f in os.listdir(input_dir) if f.endswith('.parquet')]
    size = len(file_names) * 1000
    with h5py.File(output_file, 'w') as h5f:
        images_dset = h5f.create_dataset('images', (size, 1, 150, 150), dtype='float32')
        diffs_dset = h5f.create_dataset('diffs', (size, 22500), dtype='float32')
        
        idx = 0
        for file_name in tqdm(file_names):
            X = pd.read_parquet(os.path.join(input_dir, file_name)).to_numpy()
            diffs = X[1:] - X[:-1]
            images = X.reshape(-1, 1, 150, 150)
            
            end_idx = idx + len(diffs)
            images_dset[idx:end_idx] = images[:-1]
            diffs_dset[idx:end_idx] = diffs
            
            idx = end_idx

input_dir = '../data'
output_file = '../data/images_diffs.hdf5'
convert_to_hdf5(input_dir, output_file)

100%|██████████| 984/984 [44:32<00:00,  2.72s/it]
